In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import glob
import os

from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,roc_curve,auc
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, RFE, chi2, mutual_info_classif
from sklearn import svm

from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.manifold import MDS, TSNE

In [ ]:
with open('/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat '/gdrive/My Drive/foo.txt'

In [ ]:
groups = ['class1' , 'class2']
dataset_addr = 'your address'

# Feature selection percent
tresh_percent = 0.8

# GA parameters
num_pop = 50
num_gen = 200
alpha = 0.5;
SF = 1;
CR = 0.8;
MR = 0.3;

In [ ]:
# Getting the data

temp_data = dict()
temp_label = dict()
for class_num, gr in enumerate(groups):
    gr_addr = gr

    file_list = glob.glob(dataset_addr + gr_addr + '/[!cwy]*.nii')
    img = nib.load(file_list[0])
    V = np.squeeze(np.array(img.dataobj))
    num_subj = len(file_list)
    num_raw_feat = V.size
    temp_data[gr] = np.zeros((num_subj,num_raw_feat))
    temp_label[gr] = np.zeros(num_subj,dtype='bool')
    for s_num , file in enumerate(file_list):
        img = nib.load(file)
        V = np.squeeze(np.array(img.dataobj))
        temp_data[gr][s_num,:] = V.flatten()
        temp_label[gr][s_num] = class_num


y = np.concatenate((temp_label[groups[0]],temp_label[groups[1]]))
X = np.concatenate((temp_data[groups[0]],temp_data[groups[1]]),axis=0)

In [ ]:
def plot_clustering(X_red, labels, title):
    x_min, x_max = np.min(X_red, axis=0), np.max(X_red, axis=0)
    X_red = (X_red - x_min) / (x_max - x_min)

    plt.figure(figsize=(6,3), dpi=160)
    plt.scatter(X_red[labels == True, 0], X_red[labels == True, 1],  marker='^')
    plt.scatter(X_red[labels == False, 0], X_red[labels == False, 1], marker='o')
    plt.xticks([])
    plt.yticks([])
    plt.title(title, size=17)
    plt.axis('off')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [ ]:
class feature_selector():

    def __init__(self,alldata,labels):

        self._X_full = alldata
        self._y = np.zeros(labels.shape)
        self._y[labels == True] = 1
        self.selected = np.ones(self._X_full.shape[1], dtype = np.bool)

    def t_test_paper(self,tresh_percent):

        X_in = self._X_full[:,self.selected]
        class_ind = self._y == 0
        mu_1 = np.mean(X_in[class_ind,:],axis=0)
        var_1 = np.var(X_in[class_ind,:],axis=0,ddof=1)
        n_1 = np.sum(class_ind)

        class_ind = self._y == 1
        mu_2 = np.mean(X_in[class_ind,:],axis=0)
        var_2 = np.var(X_in[class_ind,:],axis=0,ddof=1)
        n_2 = np.sum(class_ind)

        t_values = (mu_1-mu_2)/np.sqrt(var_1/n_1+var_2/n_2)
        thresh_val = tresh_percent * np.max(np.abs(t_values))
        new_idx = np.abs(t_values) > thresh_val
        self.selected[self.selected] = new_idx

        return self.selected, t_values

    def ga_paper(self,num_pop,num_gen,alpha,SF,CR,MR):

        X_train = self._X_full[:,self.selected]
        labels_train = self._y
        num_feat = X_train.shape[1]
        best_f = +np.infty
        best_f_vec = list([])
        chromo_mat = np.random.randint(2,size=(num_pop,num_feat),dtype=bool)

        for gen in range(num_gen):

            # Evaluation phase
            fitness_vec = np.array([self._calculate_scatter_matrix(X_train,labels_train,chromo_mat[chr,:],alpha) for chr in range(num_pop)])
            min_fit = np.amin(fitness_vec)
            min_ind = np.argmin(fitness_vec)
            if(min_fit < best_f):
                best_f = min_fit
                best_f_vec.append(best_f)
                best_chromo = chromo_mat[min_ind,np.newaxis]

            # Selection
            p_vec = fitness_vec**SF
            p_vec = p_vec/np.sum(p_vec)
            cum_p_vec = np.cumsum(p_vec)
            chromo_ind = np.sum(cum_p_vec < np.random.rand(num_pop -1,1),axis=1)
            chromo_mat = np.concatenate((best_chromo,chromo_mat[chromo_ind,:]),axis = 0)

            # Crossover
            for chr in range(0, num_pop, 2):
                if np.random.rand() < CR:
                    split_loc = np.random.randint(1, chromo_mat.shape[1])
                    temp = chromo_mat[chr,:split_loc].copy()
                    chromo_mat[chr,:split_loc] = chromo_mat[chr + 1,:split_loc]
                    chromo_mat[chr + 1,:split_loc] = temp

            # Mutation
            mu_inds = np.random.rand(*chromo_mat.shape) < MR
            chromo_mat[mu_inds] = 1 - chromo_mat[mu_inds]

        self.selected[self.selected] = np.squeeze(best_chromo)
        return self.selected, best_f_vec

    def chi2_method(self,keep_percent):

        X_in = self._X_full[:,self.selected]
        print(X_in.shape)
        print(X_in.min())
        print(X_in.max())

        red_num = np.ceil(X_in.shape[1] * keep_percent).astype(np.int)

        print(red_num)

        feat_selector = SelectKBest(chi2, k = red_num)
        feat_selector = feat_selector.fit(X_in, self._y)
        self.selected[self.selected] = feat_selector.get_support()
        return self.selected

    def mi_method(self,keep_percent):

        X_in = self._X_full[:,self.selected]
        red_num = np.ceil(X_in.shape[1] * keep_percent).astype(np.int)

        feat_selector = SelectKBest(mutual_info_classif, k = red_num)
        feat_selector = feat_selector.fit(X_in, self._y)
        self.selected[self.selected] = feat_selector.get_support()
        return self.selected

    def rfe_method(self,estimator, keep_percent, remove_num):

        X_in = self._X_full[:,self.selected]
        red_num = np.ceil(X_in.shape[1] * keep_percent).astype(np.int)

        feat_selector = RFE(estimator, n_features_to_select=red_num, step=remove_num)
        feat_selector = feat_selector.fit(X_in, self._y)
        self.selected[self.selected] = feat_selector.get_support()
        return self.selected

    def _calculate_scatter_matrix(self,X, labels, chromo, alpha):

        class_1_inds = labels == 0
        class_2_inds = labels == 1
        X_truncated_1 = X[np.ix_(class_1_inds,chromo)]
        X_truncated_2 = X[np.ix_(class_2_inds,chromo)]
        mu_1 = np.mean(X_truncated_1,axis=0,keepdims=True)
        mu_2 = np.mean(X_truncated_2,axis=0,keepdims=True)
        S_B = np.matmul(np.transpose(mu_1 - mu_2),(mu_1 - mu_2));
        X_til_1 = X_truncated_1 - mu_1
        X_til_2 = X_truncated_2 - mu_2
        S_W_1 = np.matmul(np.transpose(X_til_1),(X_til_1));
        S_W_2 = np.matmul(np.transpose(X_til_2),(X_til_2));
        S_W = S_W_1/X_til_1.shape[0] + S_W_2/X_til_2.shape[0]
        E = np.trace(S_W)/np.trace(S_B)
        RF = np.sum(chromo)/chromo.size
        Z = E * (1 + alpha * RF)
        return Z


In [ ]:
# Shuffle and split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# Feature selection percent for paper's method
tresh_percent = 0.8

# GA parameters for paper's method
num_pop = 50
num_gen = 200
alpha = 0.5;
SF = 1;
CR = 0.8;
MR = 0.3;

feat_sel_1 = feature_selector(X_train,y_train)
feat_idx, t_vals = feat_sel_1.t_test_paper(tresh_percent)
feat_idx, f_vecs = feat_sel_1.ga_paper(num_pop,num_gen,alpha,SF,CR,MR)

X_train_reduced = X_train[:,feat_idx]
X_test_reduced = X_test[:,feat_idx]

In [ ]:
# Our own method for featrue extraction
feat_sel_2 = feature_selector(X_train,y_train)
# feat_idx = feat_sel_2.chi2_method(keep_percent=0.1)
feat_idx = feat_sel_2.t_test_paper(tresh_percent)
feat_idx = feat_sel_2.mi_method(keep_percent=0.5)
#feat_idx, f_vecs = feat_sel_2.ga_paper(num_pop,num_gen,alpha,SF,CR,MR)
#feat_idx = feat_sel_2.rfe_method(estimator=LogisticRegression(solver='lbfgs',max_iter=1000), keep_percent=0.1, remove_num=0.01)
#feat_idx = feat_sel_2.rfe_method(estimator=LinearSVC(), keep_percent=0.1, remove_num=0.01)
print(np.sum(feat_idx))
X_train_reduced = X_train[:,feat_idx]
X_test_reduced = X_test[:,feat_idx]

In [ ]:
# Classify the data
classifier = svm.SVC(kernel="rbf")
y_score = classifier.fit(X_train_reduced, y_train).decision_function(X_test_reduced)
y_labels = y_score > 0

In [ ]:
# Compute metrics

acc_s = accuracy_score(y_test, y_labels)
f1_s = f1_score(y_test, y_labels)
pre_s = precision_score(y_test, y_labels)
recall_s = recall_score(y_test, y_labels)

print('accuracy score = ',acc_s)
print('f1 score = ',f1_s)
print('precision score = ',pre_s)
print('recall score = ',recall_s)

#Compute ROC curve and ROC area for each class
#fpr, tpr, _ = roc_curve(y_test, y_score)
#roc_auc = auc(fpr, tpr)
#print('AUC = ' , roc_auc)